In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

In [2]:
import sys
sys.path.insert(1, "../../")

In [3]:
from train import *
from data_preprocessing import *
from data_augmentation import *
from Models.darknet19 import *
import torch.optim.lr_scheduler as lr_scheduler

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
model = Darknet19(num_classes=10, device=device, dtype=torch.float32)

In [6]:
data_path = '../../../datasets/imagenette2/'
norms_path = os.path.join(data_path, 'norms.json')

In [7]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.44969913363456726, 0.44868946075439453, 0.45163223147392273]
stds are: [0.28648287057876587, 0.28796446323394775, 0.2865694761276245]


In [8]:
transformations = transforms.Compose([transforms.RandomResizedCrop((224, 224)),
                                              Augmentation(),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean=means, std=stds)])
transformations_val = transforms.Compose([transforms.Resize((224, 224)),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(mean=means, std=stds)
                                                 ])

In [9]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations_val, half=False, show=False)

In [10]:
epochs=10
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=1)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [12]:
loss_fn = nn.NLLLoss()

In [13]:
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [14]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, outputs_path='../../log/darknet19/training/')

2024-12-07 15:49:40.183470 Epoch 1 
2024-12-07 15:49:50.834510 Batch 15 
2024-12-07 15:49:58.022726 Batch 30 
2024-12-07 15:50:05.015643 Batch 45 
2024-12-07 15:50:12.731429 Batch 60 
2024-12-07 15:50:19.538197 Batch 75 
2024-12-07 15:50:26.834931 Batch 90 
2024-12-07 15:50:33.876804 Batch 105 
2024-12-07 15:50:40.614984 Batch 120 
2024-12-07 15:50:47.247678 Batch 135 
[Train] Accuracy: 26.8983%, Loss per batch: 2.0627
2024-12-07 15:51:01.635421 Batch 15 
2024-12-07 15:51:06.528626 Batch 30 
2024-12-07 15:51:12.092164 Batch 45 
2024-12-07 15:51:17.176225 Batch 60 
[Val] Accuracy: 35.2102%, loss per batch: 2.0472
Epoch 1: SGD lr 0.0100 -> 0.0090
2024-12-07 15:51:18.253939 Epoch 2 
2024-12-07 15:51:30.077576 Batch 15 
2024-12-07 15:51:39.247111 Batch 30 
2024-12-07 15:51:48.455247 Batch 45 
2024-12-07 15:51:57.723957 Batch 60 
2024-12-07 15:52:06.898526 Batch 75 
2024-12-07 15:52:16.104288 Batch 90 
2024-12-07 15:52:25.300340 Batch 105 
2024-12-07 15:52:34.530803 Batch 120 
2024-12-07 15

KeyboardInterrupt: 

In [ ]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, outputs_path='../../log/darknet19/training/', resume=True)